In [1]:
import numpy as np
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from sklearn.feature_extraction.image import extract_patches_2d
from os import listdir
from skimage import io, transform, util, img_as_float
from tqdm import tqdm_notebook
import warnings
from keras_tqdm import TQDMNotebookCallback
from PIL import Image
import numpy as np
import pandas as pd
from fastnumbers import fast_real
%matplotlib inline

/home/ivan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
import numpy as np


In [ ]:
n_train_samples = 100
n_test_samples = 20

h = 3508
w = 2480

X_train = np.zeros((n_train_samples, h, w))
Y_train = np.zeros((...))

X_test = np.zeros((n_test_samples, h, w))
Y_test = ...

path = ''

for i, filename in enumerate(os.listdir(path)[:n_train_samples]):
    img = skimage.io.imread(path + filename)
    #2grayscale
    #normalize
    X_train[i] = img
    #read txt
    # txt to mask h x w
    Y_train[i] = mask
    
#X_test
for i, filename in enumerate(os.listdir(path)[n_train_samples:n_train_samples+n_test_samples]):
    
    

In [3]:

from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization

/home/ivan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
encoding_layers = [
    Convolution2D(16, kernel, border_mode='same', input_shape=( img_h, img_w,1)),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(16, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((3,3)),

    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu')
    MaxPooling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    """Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),

    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(),"""
]

for l in autoencoder.encoding_layers:
    autoencoder.add(l)
    print(l.input_shape,l.output_shape,l)

decoding_layers = [
    """UpSampling2D(),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(512, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(256, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
"""
    UpSampling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    UpSampling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    UpSampling2D(),
    Convolution2D(64, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(32, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),

    UpSampling2D(),
    Convolution2D(16, kernel, kernel, border_mode='same'),
    BatchNormalization(),
    Activation('relu'),
    Convolution2D(n_labels, 1, 1, border_mode='valid'),
    BatchNormalization(),
]


autoencoder.add(Reshape((n_labels, img_h * img_w)))
autoencoder.add(Permute((2, 1)))
autoencoder.add(Activation('softmax'))

In [24]:
def getLinesFromFile (path):
    data = []
    f = open(path, 'r', encoding='utf-16')
    line = f.readline()
    while line:
        line = f.readline()
        if line.split("	")[0] != "" :
            num1 = line.split("	")[0]
            num2 = line.split("	")[1].split("\n")[0]
            data.append(fast_real(num1))
            data.append(fast_real(num2))
    f.close()
    return data

In [3]:
def getTextAndBackAreas (yLinesCoords):
    res = np.zeros(3508)
    i = 0
    while i < len(yLinesCoords)/2:
        j = yLinesCoords[2*i]
        while j < yLinesCoords[2*i + 1]:
            res[j] = 1
            j += 1
        i += 1   
    return res

In [4]:
def convertJpg2Png (path):
    im = Image.open(path)
    im.save(path.split('.')[0] + '.png')
    pass

In [5]:
def convert2greyscale (path):
    img = Image.open(path).convert('LA')
    img.save(path.split('.')[0] + '_greyscale.png')
    pass

In [6]:
def resizePng (path, newWidth, newHeigth):
    img = Image.open(path)
    resized_img = img.resize((newWidth, newHeigth), Image.ANTIALIAS)
    resized_img.save(path.split('.')[0] + '_resized.png')
    pass

In [7]:
def agregate2array (path, width, heigth):
    img = Image.open(path)
    pix = np.array(img)
    res = []
    for i in range(heigth):
        greySum = 0
        for j in range(width):
            greySum += pix[i][j][0]/pix[i][j][1]
        greyAverage = greySum/width
        res.append(greyAverage)
    return np.array(res)

In [8]:
path_to_lines = "582.txt"
path_to_img = "582.jpg"
width = 2480
heigth = 3508

In [9]:
yLinesCoords = getLinesFromFile (path_to_lines)
oneZeroArr = getTextAndBackAreas(yLinesCoords)

convertJpg2Png (path_to_img)
convert2greyscale (path_to_img)
greyAgrArr = agregate2array (path_to_img.split('.')[0] + '_greyscale.png', width, heigth)

In [10]:
print (oneZeroArr.shape)

(3508,)


In [11]:
print (greyAgrArr.shape)

(3508,)


In [12]:
oneZeroArr = oneZeroArr.reshape (1, -1)
greyAgrArr = greyAgrArr.reshape (1, -1)

In [13]:
print (oneZeroArr.shape)
print (greyAgrArr.shape)

(1, 3508)
(1, 3508)


In [43]:
path_to_img2 = "583.jpg"
convertJpg2Png (path_to_img2)
convert2greyscale (path_to_img2)
greyAgrArr2 = agregate2array (path_to_img2.split('.')[0] + '_greyscale.png', width, heigth)
greyAgrArr2 = greyAgrArr2.reshape(1, -1)

In [15]:
from keras.layers import Embedding, LSTM, Dense, Activation

max_features = 1000
maxlen = 3508 
batch_size = 1

model = Sequential()
model.add(Embedding(max_features, 3508, input_length=maxlen))
model.add(LSTM(3508))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam'
              )

model.fit(
    greyAgrArr, oneZeroArr, 
    batch_size=batch_size, 
    nb_epoch=1,
    verbose = 1,
    
)

result = model.predict_proba(greyAgrArr2)
print(result)

/home/ivan/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
1/1 [==============================] - 1344s 1344s/step - loss: 0.6928
[[0.49412948 0.4951485  0.49207976 ... 0.4807599  0.4933863  0.48871574]]


In [19]:
result2 = model.predict_proba(greyAgrArr2)
print(result)

False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
True
True
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


True
True
True
True
True
True
True
True
False
True
True
True
False
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
False
True
False
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False

In [44]:
yLinesCoords2 = getLinesFromFile ('583.txt')
oneZeroArr2 = getTextAndBackAreas(yLinesCoords2)

In [45]:
from sklearn.metrics import roc_auc_score, f1_score

In [46]:
roc_auc_score(oneZeroArr2, result[0])

0.952335016002836

In [47]:
f1_score(result[0] > 0.5, oneZeroArr2)

0.9229599594526102

In [48]:
model.save_weights("saved_best_model.h5")